# Redes Neurais Artificiais 2025.2

- **Disciplina**: Redes Neurais Artificiais 2025.2
- **Professora**: Elloá B. Guedes (ebgcosta@uea.edu.br)  
- **Github**: http://github.com/elloa  
        

Levando em conta a base de dados **_Forest Cover Type_**, esta parte do Projeto Prático diz respeito à proposição e avaliação de múltiplas redes neurais artificiais do tipo feedforward multilayer perceptron para o problema da classificação multi-classe da cobertura florestal em uma área do Roosevelt National Forest.

## Busca em Grade

Uma maneira padrão de escolher os parâmetros de um modelo de Machine Learning é por meio de uma busca em grade via força bruta. O algoritmo da busca em grade é dado como segue:

1. Escolha a métrica de desempenho que você deseja maximizar  
2. Escolha o algoritmo de Machine Learning (exemplo: redes neurais artificiais). Em seguida, defina os parâmetros ou hiperparâmetros deste tipo de modelo sobre os quais você dseja otimizar (número de épocas, taxa de aprendizado, etc.) e construa um array de valores a serem testados para cada parâmetro ou hiperparâmetro.  
3. Defina a grade de busca, a qual é dada como o produto cartesiano de cada parâmetro a ser testado. Por exemplo, para os arrays [50, 100, 1000] e [10, 15], tem-se que a grade é [(50,10), (50,15), (100,10), (100,15), (1000,10), (1000,15)].
4. Para cada combinação de parâmetros a serem otimizados, utilize o conjunto de treinamento para realizar uma validação cruzada (holdout ou k-fold) e calcule a métrica de avaliação no conjunto de teste (ou conjuntos de teste)
5. Escolha a combinação de parâmetros que maximizam a métrica de avaliação. Este é o modelo otimizado.

Por que esta abordagem funciona? Porque a busca em grade efetua uma pesquisa extensiva sobre as possíveis combinações de valores para cada um dos parâmetros a serem ajustados. Para cada combinação, ela estima a performance do modelo em dados novos. Por fim, o modelo com melhor métrica de desempenho é escolhido. Tem-se então que este modelo é o que melhor pode vir a generalizar mediante dados nunca antes vistos.

## Efetuando a Busca em Grade sobre Hiperparâmetros das Top-6 RNAs

Considerando a etapa anterior do projeto prático, foram identificadas pelo menos 6 melhores Redes Neurais para o problema da classificação multi-classe da cobertura florestal no conjunto de dados selecionado. Algumas destas redes possuem atributos categóricos como variáveis preditoras, enquanto outras possuem apenas os atributos numéricos como preditores.

A primeira etapa desta segunda parte do projeto consiste em trazer para este notebook estas seis arquiteturas, ressaltando:

1. Número de neurônios ocultos por camada  
2. Função de Ativação  
3. Utilização ou não de atributos categóricos   
4. Desempenho médio +- desvio padrão nos testes anteriores  
5. Número de repetições que a equipe conseguiu realizar para verificar os resultados  

Elabore uma busca em grade sobre estas arquiteturas que contemple variações nos hiperparâmetros a seguir, conforme documentação de [MLPClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html)

A. Solver  (Não usar o LBFGS, pois é mais adequado para datasets pequenos)  
B. Batch Size  
C. Learning Rate Init  
D. Paciência (n_iter_no_change)  
E. Épocas  

Nesta busca em grande, contemple a utilização do objeto [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html)

## Validação Cruzada k-fold

Na elaboração da busca em grid, vamos avaliar os modelos propostos segundo uma estratégia de validação cruzada ainda não explorada até o momento: a validação cruzada k-fold. Segundo a mesma, o conjunto de dados é particionado em k partes: a cada iteração, separa-se uma das partes para teste e o modelo é treinado com as k-1 partes remanescentes. Valores sugestivos de k na literatura são k = 3, 5 ou 10, pois o custo computacional desta validação dos modelos é alto. A métrica de desempenho é resultante da média dos desempenhos nas k iterações. A figura a seguir ilustra a ideia desta avaliação

<img src = "https://ethen8181.github.io/machine-learning/model_selection/img/kfolds.png" width=600></img>

Considerando a métrica de desempenho F1-Score, considere a validação cruzada 5-fold para aferir os resultados da busca em grande anterior.

In [5]:
!pip install -q pandas numpy matplotlib seaborn torch torchvision torchaudio kagglehub ipywidgets

In [6]:
import kagglehub

# Baixar o dataset do Kaggle
path = kagglehub.dataset_download("uciml/forest-cover-type-dataset")
print("Dataset baixado em:", path)

Using Colab cache for faster access to the 'forest-cover-type-dataset' dataset.
Dataset baixado em: /kaggle/input/forest-cover-type-dataset


In [9]:
# Cell A — locate dataset in Colab cache
import os, pprint, glob
candidates = glob.glob('/kaggle/input/forest-cover-type-dataset/**', recursive=True)
files = [p for p in candidates if os.path.isfile(p)]
print("Some files in /kaggle/input/forest-cover-type-dataset:")
pprint.pprint(files[:20])
# Common CSV path:
CSV_PATH = '/kaggle/input/forest-cover-type-dataset/covtype.csv'
if not os.path.exists(CSV_PATH):
    # fallback: find any covtype*.csv
    import glob
    found = glob.glob('/kaggle/input/**/covtype*.csv', recursive=True)
    if found:
        CSV_PATH = found[0]
    else:
        raise FileNotFoundError("covtype.csv not found in /kaggle/input; check files list above.")
print("Using CSV_PATH =", CSV_PATH)


Some files in /kaggle/input/forest-cover-type-dataset:
['/kaggle/input/forest-cover-type-dataset/covtype.csv']
Using CSV_PATH = /kaggle/input/forest-cover-type-dataset/covtype.csv


In [17]:
# Cell B — load CSV and quick inspect
import pandas as pd
CSV_PATH = '/kaggle/input/forest-cover-type-dataset/covtype.csv'  # adjust if needed
df = pd.read_csv(CSV_PATH)
print("Loaded df shape:", df.shape)
display(df.head())
print("Columns (first 20):", df.columns.tolist()[:20])
# configure sample size: None to use all data (careful!), or integer to subsample stratified
MAX_SAMPLES = None   # safe default for Colab; set to None to use full dataset if you have time/ram


Loaded df shape: (581012, 55)


,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Cover_Type
0,2596,51,3,258,0,510,221,232,148,6279,...,0,0,0,0,0,0,0,0,0,5
1,2590,56,2,212,-6,390,220,235,151,6225,...,0,0,0,0,0,0,0,0,0,5
2,2804,139,9,268,65,3180,234,238,135,6121,...,0,0,0,0,0,0,0,0,0,2
3,2785,155,18,242,118,3090,238,238,122,6211,...,0,0,0,0,0,0,0,0,0,2
4,2595,45,2,153,-1,391,220,234,150,6172,...,0,0,0,0,0,0,0,0,0,5


Columns (first 20): ['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology', 'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways', 'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm', 'Horizontal_Distance_To_Fire_Points', 'Wilderness_Area1', 'Wilderness_Area2', 'Wilderness_Area3', 'Wilderness_Area4', 'Soil_Type1', 'Soil_Type2', 'Soil_Type3', 'Soil_Type4', 'Soil_Type5', 'Soil_Type6']


In [18]:
# Cell C — prepare X, y and optional stratified sampling
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import StandardScaler

target_col = 'Cover_Type' if 'Cover_Type' in df.columns else df.columns[-1]
print("Target column:", target_col)
X = df.drop(columns=[target_col]).values.astype(np.float32)
y = df[target_col].values.astype(np.int64)
# convert labels from 1..7 to 0..6 if needed
if y.min() == 1:
    y = y - 1

print("Original dataset size:", X.shape)

if MAX_SAMPLES is not None and X.shape[0] > MAX_SAMPLES:
    print(f"Stratified sampling to {MAX_SAMPLES} samples...")
    sss = StratifiedShuffleSplit(n_splits=1, test_size=MAX_SAMPLES, random_state=42)
    for _, idx in sss.split(X, y):
        X_sample = X[idx]
        y_sample = y[idx]
else:
    X_sample, y_sample = X, y

print("Using sample shape:", X_sample.shape, "n_classes:", len(np.unique(y_sample)))

Target column: Cover_Type
Original dataset size: (581012, 54)
Using sample shape: (581012, 54) n_classes: 7


In [19]:
# Cell D — PyTorch helpers
import torch, gc
from torch import nn
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import f1_score, classification_report, confusion_matrix

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Device:", device)

class NumpyDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.from_numpy(X).float()
        self.y = torch.from_numpy(y).long()
    def __len__(self):
        return len(self.y)
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dim, n_classes, dropout=0.2):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, max(hidden_dim//2, 8)),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(max(hidden_dim//2, 8), n_classes)
        )
    def forward(self, x):
        return self.net(x)

def train_one_epoch(model, loader, optimizer, criterion):
    model.train()
    total_loss = 0.0
    for Xb, yb in loader:
        Xb, yb = Xb.to(device), yb.to(device)
        optimizer.zero_grad()
        logits = model(Xb)
        loss = criterion(logits, yb)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * Xb.size(0)
    return total_loss / len(loader.dataset)

def eval_model(model, loader):
    model.eval()
    ys, ps = [], []
    with torch.no_grad():
        for Xb, yb in loader:
            Xb = Xb.to(device)
            logits = model(Xb)
            preds = logits.argmax(dim=1).cpu().numpy()
            ps.append(preds)
            ys.append(yb.numpy())
    y_true = np.concatenate(ys)
    y_pred = np.concatenate(ps)
    return float(f1_score(y_true, y_pred, average='weighted')), y_true, y_pred


Device: cuda


In [20]:
# Cell E — grid search manual (small grid); adjust param_grid for more experiments
import time, itertools, json
from copy import deepcopy
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
import gc

n_classes = len(np.unique(y_sample))
input_dim = X_sample.shape[1]
print("input_dim:", input_dim, "n_classes:", n_classes)

param_grid = {
    'hidden_dim': [128, 256],
    'lr': [1e-3, 1e-4],
    'batch_size': [256],
    'epochs': [8]
}

def iter_grid(grid):
    keys = list(grid.keys())
    for vals in itertools.product(*(grid[k] for k in keys)):
        yield dict(zip(keys, vals))

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
best_cfg, best_score = None, -1.0
results = []

for cfg in iter_grid(param_grid):
    fold_scores = []
    t0 = time.time()
    print("Testing cfg:", cfg)
    for fold_idx, (train_idx, val_idx) in enumerate(skf.split(X_sample, y_sample), 1):
        # fit scaler on train fold
        scaler = StandardScaler()
        X_train_fold = scaler.fit_transform(X_sample[train_idx])
        X_val_fold = scaler.transform(X_sample[val_idx])
        y_train_fold = y_sample[train_idx]
        y_val_fold = y_sample[val_idx]

        train_ds = NumpyDataset(X_train_fold, y_train_fold)
        val_ds = NumpyDataset(X_val_fold, y_val_fold)
        train_loader = DataLoader(train_ds, batch_size=cfg['batch_size'], shuffle=True)
        val_loader = DataLoader(val_ds, batch_size=cfg['batch_size'], shuffle=False)

        model = MLP(input_dim=input_dim, hidden_dim=cfg['hidden_dim'], n_classes=n_classes).to(device)
        opt = torch.optim.Adam(model.parameters(), lr=cfg['lr'])
        criterion = nn.CrossEntropyLoss()

        for ep in range(cfg['epochs']):
            _ = train_one_epoch(model, train_loader, opt, criterion)

        f1_val, _, _ = eval_model(model, val_loader)
        fold_scores.append(f1_val)

        # cleanup
        del model, opt, criterion, train_loader, val_loader, train_ds, val_ds
        torch.cuda.empty_cache()
        gc.collect()

    mean_f1 = float(np.mean(fold_scores))
    elapsed = time.time() - t0
    print(f"  Mean F1 5-fold = {mean_f1:.4f} (time {elapsed:.1f}s)")
    results.append((cfg, mean_f1))
    if mean_f1 > best_score:
        best_score = mean_f1
        best_cfg = deepcopy(cfg)

print("\nBest config:", best_cfg, "best_cv_f1 =", best_score)
# save results
with open('/content/grid_search_results.json', 'w') as f:
    json.dump({'best': best_cfg, 'best_score': best_score, 'results': [[r[0], r[1]] for r in results]}, f, indent=2)
print("Saved grid search summary to /content/grid_search_results.json")

input_dim: 54 n_classes: 7
Testing cfg: {'hidden_dim': 128, 'lr': 0.001, 'batch_size': 256, 'epochs': 8}
  Mean F1 5-fold = 0.8342 (time 234.5s)
Testing cfg: {'hidden_dim': 128, 'lr': 0.0001, 'batch_size': 256, 'epochs': 8}
  Mean F1 5-fold = 0.7591 (time 233.1s)
Testing cfg: {'hidden_dim': 256, 'lr': 0.001, 'batch_size': 256, 'epochs': 8}
  Mean F1 5-fold = 0.8650 (time 233.8s)
Testing cfg: {'hidden_dim': 256, 'lr': 0.0001, 'batch_size': 256, 'epochs': 8}
  Mean F1 5-fold = 0.7837 (time 233.1s)

Best config: {'hidden_dim': 256, 'lr': 0.001, 'batch_size': 256, 'epochs': 8} best_cv_f1 = 0.8649654852949569
Saved grid search summary to /content/grid_search_results.json


In [21]:
# Cell F — train final on X_sample with scaler + save model & scaler
import joblib
from sklearn.preprocessing import StandardScaler
best = best_cfg
print("Training final with:", best)

scaler_final = StandardScaler()
X_scaled = scaler_final.fit_transform(X_sample)
y_final = y_sample

final_ds = NumpyDataset(X_scaled, y_final)
final_loader = DataLoader(final_ds, batch_size=best['batch_size'], shuffle=True)

model_final = MLP(input_dim=input_dim, hidden_dim=best['hidden_dim'], n_classes=n_classes).to(device)
optimizer = torch.optim.Adam(model_final.parameters(), lr=best['lr'])
criterion = nn.CrossEntropyLoss()

for epoch in range(best['epochs']):
    loss = train_one_epoch(model_final, final_loader, optimizer, criterion)
    print(f"Epoch {epoch+1}/{best['epochs']} - loss: {loss:.4f}")

MODEL_PATH = '/content/cover_mlp_best.pth'
SCALER_PATH = '/content/cover_scaler.pkl'
torch.save(model_final.state_dict(), MODEL_PATH)
joblib.dump(scaler_final, SCALER_PATH)
print("Saved model to", MODEL_PATH)
print("Saved scaler to", SCALER_PATH)

# save summary
summary = {
    'n_total_available': int(X.shape[0]),
    'n_used_sample': int(X_sample.shape[0]),
    'best_cfg': best,
    'best_cv_f1': best_score,
    'model_path': MODEL_PATH,
    'scaler_path': SCALER_PATH
}
import json
with open('/content/cover_training_summary.json', 'w') as f:
    json.dump(summary, f, indent=2)
print("Saved training summary to /content/cover_training_summary.json")


Training final with: {'hidden_dim': 256, 'lr': 0.001, 'batch_size': 256, 'epochs': 8}
Epoch 1/8 - loss: 0.5936
Epoch 2/8 - loss: 0.4880
Epoch 3/8 - loss: 0.4481
Epoch 4/8 - loss: 0.4226
Epoch 5/8 - loss: 0.4046
Epoch 6/8 - loss: 0.3909
Epoch 7/8 - loss: 0.3794
Epoch 8/8 - loss: 0.3691
Saved model to /content/cover_mlp_best.pth
Saved scaler to /content/cover_scaler.pkl
Saved training summary to /content/cover_training_summary.json


## Identificando a mellhor solução

Como resultado da busca em grande com validação cruzada 5-fold, identifique o modelo otimizado com melhor desempenho para o problema. Apresente claramente este modelo, seus parâmetros, hiperparâmetros otimizados e resultados para cada um dos folds avaliados. Esta é a melhor solução identificada em decorrência deste projeto

Holdout F1-weighted: 0.7585472690498573

Classification report:
               precision    recall  f1-score   support

           0     0.7747    0.7428    0.7584      1458
           1     0.7809    0.8350    0.8070      1951
           2     0.6721    0.8415    0.7473       246
           3     0.5833    0.3684    0.4516        19
           4     1.0000    0.0769    0.1429        65
           5     0.4861    0.2917    0.3646       120
           6     0.8403    0.7092    0.7692       141

    accuracy                         0.7665      4000
   macro avg     0.7339    0.5522    0.5773      4000
weighted avg     0.7678    0.7665    0.7585      4000



,pred_0,pred_1,pred_2,pred_3,pred_4,pred_5,pred_6
true_0,1083,355,0,0,0,2,18
true_1,274,1629,25,0,0,22,1
true_2,0,24,207,4,0,11,0
true_3,0,0,10,7,0,2,0
true_4,0,59,1,0,5,0,0
true_5,1,18,65,1,0,35,0
true_6,40,1,0,0,0,0,100


Saved confusion matrix to /content/confusion_matrix_holdout.csv


## Empacotando a solução

Suponha que você deve entregar este classificador ao órgão responsável por administrar o Roosevelt National Park. Para tanto, você deve fazer uma preparação do mesmo para utilização neste cenário. Uma vez que já identificou os melhores parâmetros e hiperparâmetros, o passo remanescente consiste em treinar o modelo com estes valores e todos os dados disponíveis, salvando o conjunto de pesos do modelo ao final para entrega ao cliente. Assim, finalize o projeto prático realizando tais passos.

1. Consulte a documentação a seguir:
https://scikit-learn.org/stable/modules/model_persistence.html  
2. Treine o modelo com todos os dados  
3. Salve o modelo em disco  
4. Construa uma rotina que recupere o modelo em disco  
5. Mostre que a rotina é funcional, fazendo previsões com todos os elementos do dataset e exibindo uma matriz de confusão das mesmas